In [1]:
import pandas as pd
import numpy as np 
import json
import time
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

import datetime
import dateutil.parser

import seaborn as sns
import matplotlib.pyplot as plt

import ast
import pytz
import math

import chart_studio.plotly as py
import plotly.express as px
import cufflinks as cf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from functools import reduce

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected = True)
cf.go_offline()

%matplotlib inline


In [2]:
def get_app(x):
    x = x["device"]
    return x["app"]



def get_version(x):
    x = x["device"]
    return x["ver"]

def get_platform(x):
    x = x["device"]
    return x["platform"]


def get_substatus(x):
    x = x["user"]
    return x["subStatus"]


In [3]:

df_0613 = pd.read_csv(r"/Users/xiaoyuchen/Desktop/std_data/app_event_0612_0615/appevent_2021-06-13.csv.zip", index_col = None)
df_0614 = pd.read_csv(r"/Users/xiaoyuchen/Desktop/std_data/app_event_0612_0615/appevent_2021-06-14.csv.zip", index_col = None)
df_0615 = pd.read_csv(r"/Users/xiaoyuchen/Desktop/std_data/app_event_0612_0615/appevent_2021-06-15.csv.zip", index_col = None)

df_week = pd.concat(
    [
     df_0613,
     df_0614,
     df_0615
    ]
)


In [4]:

t1 = time.time()

meta = []


for m in df_week["meta"]:
    m1 = m.replace("'", "\"").replace("True", "true").replace("False", 'false')
    m2 = json.loads(m1)
    meta.append(m2)
    
    
df_week["meta"] = meta
df_week["ver"] = df_week["meta"].apply(get_version)
    
df_2460 = df_week[df_week["ver"] == '24.6.0']

df_2460["app"] = df_2460["meta"].apply(get_app)
df_2460["platform"] = df_2460["meta"].apply(get_platform)
df_2460["sub_status"] = df_2460["meta"].apply(get_substatus)

df_fg_2460 = df_2460[df_2460["app"] == "FG"]
# fg_2460_ios = df_fg_2460[df_fg_2460["platform"] == "ios"]

t2 = time.time()
print(t2 - t1)

77.3746223449707


## Onboarding

In [5]:
fg_onboard = df_fg_2460[df_fg_2460["event"].str.startswith(("Enter", "Tap", "Pass", "Exit", "Fail", "Complete"))]
fg_onboard.sort_values(by = ["transaction", "stamp"], inplace = True)

In [6]:
fg_onboard_ios = fg_onboard[fg_onboard["platform"] == "ios"]

In [7]:
oboard_event_count = fg_onboard_ios[["transaction", "event"]].groupby([
    "event"
]).count().reset_index()

event_dict = {"EnterOnboard":0,
              "EnterWelcome":1,
              "TapClubSignin":2,
              "TapFirstTimeHere":3,
              "TapExistingUser":4,
              "EnterBenefits":5,
              "TapProceed":6,
              "EnterClaimFreeTrial":7,
              "TapStartFreeTrial":8,
              "TapCancelClaimFreeTrial":9,
              "EnterChildlock":10,
              "TapCancelChildlock":11,
              "PassChildlock":12,
              "FailChildlock":13,
              "EnterShop":14,
              "TapPayMonthly":15,
              "TapPayAnnually":16,
              "TapCancelShop":17,
              "TapRestorePurchases":18,
              "EnterConfirmPurchase":19,
              "CompleteMonthlyPurchase":20,
              "CompleteAnnualPurchase":21,
              "EnterRegister":22,
              "TapRegisterNow":23,
              "TapCancelRegister":24,
              "EnterSelectStructure":25,
              "TapStructured":26,
              "TapUnstructured":27,
              "EnterPushNotificationPrimer":28,
              "TapGotIt":29,
              "ExitOnboard":30
             }
              


oboard_event_count.sort_values(by = ["event"],
                 key = lambda x: x.map(event_dict), 
                 inplace = True)
oboard_event_count.rename(columns = {"transaction":"count"}, inplace = True)
oboard_event_count





,event,count
5,EnterOnboard,403
10,EnterWelcome,95
19,TapFirstTimeHere,71
18,TapExistingUser,14
2,EnterBenefits,144
23,TapProceed,71
3,EnterClaimFreeTrial,79
26,TapStartFreeTrial,45
15,TapCancelClaimFreeTrial,31
14,TapCancelChildlock,14


In [216]:
# oboard_event_count_and.to_csv(r"/Users/xiaoyuchen/Desktop/std_data/app_event_0612_0615/fg_onboard_count_and.csv")

In [8]:
# x = fg_onboard_ios[["transaction","event", "stamp"]].groupby(["transaction","event"]).count().reset_index()
# x.sort_values(by = "stamp", ascending = False, inplace = True)
# x[(x["stamp"]>1) & (x["event"] == "EnterBenefits")]
